<a href="https://colab.research.google.com/github/Vedmani/wear-particle-classification/blob/main/Xception_hmm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install wandb -qU
!wandb login

     |████████████████████████████████| 1.9 MB 38.7 MB/s 
     |████████████████████████████████| 162 kB 68.5 MB/s 
     |████████████████████████████████| 182 kB 69.8 MB/s 
     |████████████████████████████████| 63 kB 1.3 MB/s 
     |████████████████████████████████| 162 kB 73.2 MB/s 
     |████████████████████████████████| 158 kB 64.3 MB/s 
     |████████████████████████████████| 157 kB 74.5 MB/s 
     |████████████████████████████████| 157 kB 77.7 MB/s 
     |████████████████████████████████| 157 kB 77.8 MB/s 
     |████████████████████████████████| 157 kB 79.7 MB/s 
     |████████████████████████████████| 157 kB 72.4 MB/s 
     |████████████████████████████████| 157 kB 72.3 MB/s 
     |████████████████████████████████| 157 kB 64.2 MB/s 
     |████████████████████████████████| 156 kB 68.7 MB/s 
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wand

In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import wandb
from wandb.keras import WandbCallback

In [4]:
tf.keras.utils.set_random_seed(123)
tf.config.experimental.enable_op_determinism()

In [5]:
PATH = '/content/drive/MyDrive/Research Internship/wear particles'
batch_size = 32
img_height = 224
img_width = 224

train_ds = tf.keras.utils.image_dataset_from_directory(
  PATH,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.utils.image_dataset_from_directory(
  PATH,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

class_names = train_ds.class_names
print(class_names)
num_classes = len(class_names)

Found 740 files belonging to 5 classes.
Using 592 files for training.
Found 740 files belonging to 5 classes.
Using 148 files for validation.
['cutting wear', 'normal rubbing and cutting wear combined', 'normal rubbing wear', 'red oxide', 'severe sliding wear']


In [11]:
xception = tf.keras.applications.Xception(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling=None)
data_augmentation = tf.keras.Sequential([layers.RandomFlip("horizontal_and_vertical"),layers.RandomRotation(0.2),])
model_xception=Sequential([
  #data_augmentation,
  layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  xception,
  layers.Flatten(),
  layers.Dense(32, activation='relu'),
  #layers.Dropout(0.5), 
  layers.Dense(num_classes, activation='softmax')
])
xception.trainable=True
model_xception.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_1 (Rescaling)     (None, 224, 224, 3)       0         
                                                                 
 xception (Functional)       (None, None, None, 2048)  20861480  
                                                                 
 flatten_1 (Flatten)         (None, 100352)            0         
                                                                 
 dense_2 (Dense)             (None, 32)                3211296   
                                                                 
 dense_3 (Dense)             (None, 5)                 165       
                                                                 
Total params: 24,072,941
Trainable params: 24,018,413
Non-trainable params: 54,528
_________________________________________________________________


In [12]:
model_xception.compile(
              optimizer= tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False, name='Adam'),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy']
              )

In [13]:
wandb.init(
      # Set the project where this run will be logged
      project="Transfer_learning", 
      name = "Xception_final",
      notes = "without augment and without dropout"
)

In [14]:
%%time
epochs=40
history = model_xception.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs,
callbacks=[WandbCallback()]
)

Epoch 1/40
 6/19 [========>.....................] - ETA: 8s - loss: 2.9514 - accuracy: 0.4531

19/19 [==============================] - ETA: 0s - loss: 1.9841 - accuracy: 0.5034

wandb: Adding directory to artifact (/content/wandb/run-20221018_060621-242d7ufv/files/model-best)... Done. 1.8s


19/19 [==============================] - 45s 2s/step - loss: 1.9841 - accuracy: 0.5034 - val_loss: 4.1155 - val_accuracy: 0.4932
Epoch 2/40
19/19 [==============================] - ETA: 0s - loss: 0.5944 - accuracy: 0.7804

wandb: Adding directory to artifact (/content/wandb/run-20221018_060621-242d7ufv/files/model-best)... Done. 1.9s


...

19/19 [==============================] - 35s 2s/step - loss: 0.0417 - accuracy: 0.9932 - val_loss: 0.0755 - val_accuracy: 0.9865
Epoch 23/40
19/19 [==============================] - 14s 706ms/step - loss: 0.0451 - accuracy: 0.9899 - val_loss: 0.5456 - val_accuracy: 0.9865
Epoch 24/40
19/19 [==============================] - 14s 713ms/step - loss: 0.0098 - accuracy: 0.9983 - val_loss: 0.1744 - val_accuracy: 0.9865
Epoch 25/40
19/19 [==============================] - ETA: 0s - loss: 0.0427 - accuracy: 0.9882

wandb: Adding directory to artifact (/content/wandb/run-20221018_060621-242d7ufv/files/model-best)... Done. 1.9s


19/19 [==============================] - 37s 2s/step - loss: 0.0427 - accuracy: 0.9882 - val_loss: 0.0319 - val_accuracy: 0.9932
Epoch 26/40
19/19 [==============================] - ETA: 0s - loss: 0.0054 - accuracy: 0.9983

wandb: Adding directory to artifact (/content/wandb/run-20221018_060621-242d7ufv/files/model-best)... Done. 1.7s


19/19 [==============================] - 36s 2s/step - loss: 0.0054 - accuracy: 0.9983 - val_loss: 0.0282 - val_accuracy: 0.9932
Epoch 27/40
19/19 [==============================] - 14s 687ms/step - loss: 0.0143 - accuracy: 0.9932 - val_loss: 0.3370 - val_accuracy: 0.9459
Epoch 28/40
19/19 [==============================] - 14s 719ms/step - loss: 0.0198 - accuracy: 0.9966 - val_loss: 0.1569 - val_accuracy: 0.9865
Epoch 29/40
19/19 [==============================] - 14s 714ms/step - loss: 0.0060 - accuracy: 0.9983 - val_loss: 0.0656 - val_accuracy: 0.9932
Epoch 30/40
19/19 [==============================] - 14s 688ms/step - loss: 0.0248 - accuracy: 0.9966 - val_loss: 0.2896 - val_accuracy: 0.9459
Epoch 31/40
19/19 [==============================] - 13s 679ms/step - loss: 0.3916 - accuracy: 0.9578 - val_loss: 1.4176 - val_accuracy: 0.8851
Epoch 32/40
19/19 [==============================] - 14s 699ms/step - loss: 0.1042 - accuracy: 0.9814 - val_loss: 4.6107 - val_accuracy: 0.8243
Epoch 3

In [15]:
wandb.finish()

accuracy,▁▅▆▇██▇█▇▇█▇██▇▇██████████████▇█████████
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▃▂▂▁▂▂▁▂▂▂▁▂▁▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▂▁▂▁
val_accuracy,▁▄▅▄▆▅▆▅▇▂▆▄▅▃▄▆▅▇████████▇██▇▆▆▇▇█▇██▆▄
val_loss,▂▂▂▃▂▃▂▂▁▆▂▂▂▂▇▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▂█
accuracy,0.96959
best_epoch,25
best_val_loss,0.0282
epoch,39
loss,0.1132
val_accuracy,0.68243
